In [ ]:
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
import GenSyntheticMNSITFixedWidthModule as GenDataset
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Consts
DATSET_SIZE = 100000
WIDTH_NUMS = 2

In [ ]:
def dense_to_one_hot(labels_dense, num_classes=10):
    """Convert class labels from scalars to one-hot vectors."""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

In [ ]:
# Get the dataset
X, Y = GenDataset.getDataSet(WIDTH_NUMS, DATSET_SIZE)
X = X.reshape([-1, 28, 28 * WIDTH_NUMS, 1])
Y = [dense_to_one_hot(Y[:,idx]) for idx in range(Y.shape[1])] 

In [ ]:
print (Y[0].shape)

In [ ]:
# Test a sample data
%matplotlib inline
idx = np.random.randint(0,DATSET_SIZE)
print ([Y[i][idx] for i in range(len(Y))])
print (X[idx].shape)
plt.imshow(np.squeeze(X[idx]), cmap = 'gray')

In [ ]:
# Building convolutional network
network = input_data(shape=[None, 28, 28 * WIDTH_NUMS, 1], name='input')
network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = fully_connected(network, 128, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 256, activation='tanh')
network = dropout(network, 0.8)
softmax1 = fully_connected(network, 10, activation='softmax')
softmax2 = fully_connected(network, 10, activation='softmax')
network1 = regression(softmax1, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target1')
network2 = regression(softmax2, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target2')
network = tflearn.merge([network1, network2], mode='elemwise_sum')

In [ ]:
model = tflearn.DNN(network, tensorboard_verbose=1)
model.fit({'input': X}, {'target1': Y[0], 'target2': Y[1]}, n_epoch=5, snapshot_step=100, show_metric=True, run_id='convnet_mnist')